# Search ~500,000 metagenomes for matches to a genome sequence

See [the GitHub repo](https://github.com/sourmash-bio/2022-search-sra-with-mastiff) for more info!

The mastiff server is unpublished work by Luiz Irber - please [file an issue](https://github.com/dib-lab/sourmash/issues) if you are interested in how to cite it.

## Python miscellany

Import all the Python libraries we need to use

In [1]:
import pandas as pd
import os
import sourmash, screed
import io
import urllib3

## Download SRA metadata information.

This is a precomputed CSV that we download once per instance; it's used to summarize the `ScientificName` (i.e. metagenome type) of matches at the bottom.

In [2]:
!wget --no-clobber https://osf.io/download/762mk/ -O sra.runinfo.csv.gz

--2023-02-02 09:14:40--  https://osf.io/download/762mk/
Resolving osf.io (osf.io)... 35.190.84.173
Connecting to osf.io (osf.io)|35.190.84.173|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://files.osf.io/v1/resources/v5e7t/providers/osfstorage/630bb6d2f7c12c3bae91d02a [following]
--2023-02-02 09:14:40--  https://files.osf.io/v1/resources/v5e7t/providers/osfstorage/630bb6d2f7c12c3bae91d02a
Resolving files.osf.io (files.osf.io)... 35.186.214.196
Connecting to files.osf.io (files.osf.io)|35.186.214.196|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://storage.googleapis.com/cos-osf-prod-files-us/b15ce5352b62bcb071d8e4cff6f7c0b21e18eb1e45ad32592bcc1481ad1e6199?response-content-disposition=attachment%3B%20filename%3D%22sra.runinfo.csv.gz%22%3B%20filename%2A%3DUTF-8%27%27sra.runinfo.csv.gz&GoogleAccessId=files-us%40cos-osf-prod.iam.gserviceaccount.com&Expires=1675358141&Signature=Y%2FipUBx90xgR7I4XpzQcTFUT3OYeScrKrAL

## Load the SRA metadata into a pandas dataframe

In [3]:
run_info = pd.read_csv('sra.runinfo.csv.gz')
print(f"Loaded {len(run_info)} records from sra.runinfo.csv.gz")
run_info.head()

Loaded 702013 records from sra.runinfo.csv.gz


,Unnamed: 0,Run,ScientificName
0,0,SRR18036904,bovine metagenome
1,1,SRR18036905,bovine metagenome
2,2,SRR18036906,bovine metagenome
3,3,SRR18036907,bovine metagenome
4,4,SRR18036908,bovine metagenome


## Generate a sourmash sketch for a query

In order to query mastiff with a genome, we need to sketch the genome using [sourmash](https://sourmash.readthedocs.io/). Here use the Python interface for sourmash.

The below code loads all of the sequence data using [the screed library](https://screed.readthedocs.io/), and feeds it into a sourmash [FracMinHash sketch](https://dib-lab.github.io/2020-paper-sourmash-gather/).

You can replace the `QUERY_SEQUENCE_FILE` with your own query filename, if you like. It should be over 10kb in size (ideally you want ~5-10 hashes to result in order for the query to be robust).

In [53]:
# generate sourmash sketch
#QUERY_SEQUENCE_FILE='sequences/shewanella.fa.gz'
QUERY_SEQUENCE_FILE='../00_data/ho_et_al_2019/genomes/Staphylococcus_aureus_GCF_000013425.1_ASM1342v1_genomic.fna.gz'


total_bp = 0

sketch = sourmash.MinHash(0, 21, scaled=1000)
with screed.open(QUERY_SEQUENCE_FILE) as records:
    for record in records:
        sketch.add_sequence(record.sequence, force=True)
        total_bp += len(record.sequence)
        
print(f"generated {len(sketch)} hashes by sketching {total_bp:g} bp from '{QUERY_SEQUENCE_FILE}'")


generated 2799 hashes by sketching 2.82136e+06 bp from '../00_data/ho_et_al_2019/genomes/Staphylococcus_aureus_GCF_000013425.1_ASM1342v1_genomic.fna.gz'


## Serialize the sourmash sketch

Now, we convert the sourmash sketch into gzipped JSON bytes.

In [54]:
# serialize to SourmashSignature / gzipped JSON
ss = sourmash.SourmashSignature(sketch)

buf = io.BytesIO()
sourmash.save_signatures([ss], buf, compression=True)
print(f"serialized sourmash signature into {len(buf.getvalue())} bytes.")

serialized sourmash signature into 23564 bytes.


## Query the mastiff server

Next, use `urllib3` to contact the mastiff server, send in the sourmash signature information, and get the results.

In [55]:
# query mastiff
http = urllib3.PoolManager()
r = http.request('POST',
                 'https://mastiff.sourmash.bio/search',
                 body = buf.getvalue(),
                 headers = { 'Content-Type': 'application/json'})

query_results_text = r.data.decode('utf-8')

## Parse the mastiff results as CSV

mastiff returns the results in CSV format, with two columns - the SRA accession, and the containment index of the query.

The containment index is calculated as the number of hashes in the query that are present in a matching metagenome, divided by the total number of hashes in the query. A containment of 1.0 means every hash in the query sketch is present, and estimates that the original query sequence is entirely present; a containment of 0.5 means that 50% of the query sketch is present, suggesting that only about half of the query sequence is actually in the metagenome.

In [56]:
# load results into a pandas data frame
results_wrap_fp = io.StringIO(query_results_text)
mastiff0_df = pd.read_csv(results_wrap_fp)

print(f"Loaded {len(mastiff0_df)} mastiff results into a dataframe!")

Loaded 39573 mastiff results into a dataframe!


## Explore the results

Look at the best and worst matches.

In [57]:
mastiff0_df.head()

,SRA accession,containment
0,SRR12829165,0.997142
1,SRR14116371,0.983566
2,SRR14116367,0.982494
3,SRR12829164,0.982136
4,SRR14116368,0.981065


In [58]:
mastiff0_df.tail()

,SRA accession,containment
39568,SRR925111,0.017864
39569,SRR5395414,0.017864
39570,SRR16841587,0.017864
39571,SRR17635709,0.017864
39572,SRR5903385,0.017864


## Discard matches below a containment of 0.2

Per [our work on containment ANI](https://github.com/sourmash-bio/sourmash/issues/1859), this corresponds to a minimum of 92% average nucleotide identity.

In [59]:
THRESHOLD=0.2

mastiff_df = mastiff0_df[mastiff0_df['containment'] >= THRESHOLD]
print(f"Filtering results at a containment of >= {THRESHOLD:.2f}; {len(mastiff_df)} of {len(mastiff0_df)} left.")

Filtering results at a containment of >= 0.20; 5830 of 39573 left.


## Explore metagenome "types" through `Scientific Name`

Now, correlate results with the `ScientificName` metadata, which records which "kind" of metagenome this is.

In [60]:
mastiff2_df = mastiff_df.set_index('SRA accession').join(run_info.set_index('Run')['ScientificName'])
mastiff2_df.head()

,containment,ScientificName
SRA accession,,
SRR12829165,0.997142,synthetic metagenome
SRR14116371,0.983566,NaN
SRR14116367,0.982494,NaN
SRR12829164,0.982136,synthetic metagenome
SRR14116368,0.981065,NaN


In [61]:
# how many have 'null' scientific name?
null_df = mastiff2_df[mastiff2_df['ScientificName'].isnull()]
print(len(null_df))

752


In [62]:
mastiff3_df = mastiff2_df[~mastiff2_df['ScientificName'].isnull()]
print(f"Of {len(mastiff2_df)} MAGsearch results, {len(mastiff3_df)} have non-null ScientificName")
mastiff3_df.head()

Of 5830 MAGsearch results, 5078 have non-null ScientificName


,containment,ScientificName
SRA accession,,
SRR12829165,0.997142,synthetic metagenome
SRR12829164,0.982136,synthetic metagenome
SRR13855169,0.974991,synthetic metagenome
SRR13855170,0.974991,synthetic metagenome
SRR13773701,0.973562,human gut metagenome


## Group and aggregate by metagenome type

Explore which types of metagenomes tend to contain this query.

In [63]:
# what are the top ScientificNames of the matches?
mastiff3_df["ScientificName"].value_counts()[:20]

metagenome                         1303
human gut metagenome               1100
gut metagenome                      552
human skin metagenome               421
human metagenome                    284
synthetic metagenome                192
human nasopharyngeal metagenome     184
pig gut metagenome                  163
human lung metagenome               130
wastewater metagenome               122
respiratory tract metagenome         72
Homo sapiens                         69
skin metagenome                      58
blood metagenome                     46
indoor metagenome                    40
mixed culture metagenome             30
mouse gut metagenome                 26
terrestrial metagenome               19
feces metagenome                     16
lung metagenome                      13
Name: ScientificName, dtype: int64